In [1]:
import numpy as np
from pathlib import Path

from keckdata import KeckData, KeckDataList, fits_reader, analysis

/Users/jwalawender/anaconda/lib/python3.6/site-packages/scipy/__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.11.3)
  UserWarning)


In [2]:
p1 = Path('/Volumes/JOSHW/HIRES/2019oct11')
p2 = Path('/Volumes/JOSHW/HIRES/2019oct12')

In [3]:
files = sorted([f for f in p1.glob('*fits')])
files.extend( sorted([f for f in p2.glob('*fits')]) )
kdl = KeckDataList(files, verbose=False)

In [4]:
mb = analysis.make_master_bias(kdl)

2019-10-14 15:07:50     INFO: Making master bias from 13 frames
2019-10-14 15:07:50     INFO: Making master bias for each of 3 extensions
2019-10-14 15:08:00    DEBUG:   Master Bias 0 (mean, med, mode, std) = 920.4, 920.4, 920, 0.53
2019-10-14 15:08:10    DEBUG:   Master Bias 1 (mean, med, mode, std) = 974.9, 974.8, 975, 0.56
2019-10-14 15:08:20    DEBUG:   Master Bias 2 (mean, med, mode, std) = 906.9, 906.8, 907, 0.64
2019-10-14 15:08:20     INFO:   Done


In [5]:
rn = analysis.determine_read_noise(kdl, master_bias=mb)

2019-10-14 15:08:20     INFO: Determining read noise
2019-10-14 15:08:20     INFO:   Checking that all inputs are BIAS frames
2019-10-14 15:08:20     INFO:   Found 7 biases
2019-10-14 15:08:20     INFO: Making master bias from 6 frames
2019-10-14 15:08:20     INFO: Making master bias for each of 3 extensions
2019-10-14 15:08:28    DEBUG:   Master Bias 0 (mean, med, mode, std) = 919.6, 919.7, 920, 0.61
2019-10-14 15:08:35    DEBUG:   Master Bias 1 (mean, med, mode, std) = 973.6, 973.7, 974, 0.58
2019-10-14 15:08:42    DEBUG:   Master Bias 2 (mean, med, mode, std) = 906.3, 906.3, 906, 0.62
2019-10-14 15:08:42     INFO:   Done
2019-10-14 15:08:42     INFO:   Determining read noise for each of 3 extensions
2019-10-14 15:08:46    DEBUG:   Bias Diff (mean, med, mode, std) = 0.2, 0.2, 0, 1.43
2019-10-14 15:08:46     INFO:   Read Noise is 1.32 adu for extension 1
2019-10-14 15:08:50    DEBUG:   Bias Diff (mean, med, mode, std) = 0.2, 0.2, 0, 1.37
2019-10-14 15:08:50     INFO:   Read Noise is 1

In [6]:
rn

<Quantity [ 1.32449115, 1.26762389, 1.30709388] adu>

In [ ]:
dc = analysis.determine_dark_current(kdl, master_bias=mb)

2019-10-14 15:08:54     INFO: Determining dark current
2019-10-14 15:08:54     INFO:   Found 6 dark frames
2019-10-14 15:08:54     INFO:   Determining dark current for each of 3 extensions
2019-10-14 15:08:58    DEBUG:   Bias Diff (mean, med, std) = 0.8, 0.8, 1.40
2019-10-14 15:09:01    DEBUG:   Bias Diff (mean, med, std) = 0.9, 0.8, 1.23
2019-10-14 15:09:05    DEBUG:   Bias Diff (mean, med, std) = 0.6, 0.5, 1.26
2019-10-14 15:09:08    DEBUG:   Bias Diff (mean, med, std) = 1.0, 1.0, 1.41
2019-10-14 15:09:12    DEBUG:   Bias Diff (mean, med, std) = 1.2, 1.2, 1.25
2019-10-14 15:09:15    DEBUG:   Bias Diff (mean, med, std) = 0.4, 0.4, 1.28
2019-10-14 15:09:19    DEBUG:   Bias Diff (mean, med, std) = 0.7, 0.7, 1.41
2019-10-14 15:09:23    DEBUG:   Bias Diff (mean, med, std) = 1.1, 1.2, 1.24
2019-10-14 15:09:26    DEBUG:   Bias Diff (mean, med, std) = 0.3, 0.3, 1.26
2019-10-14 15:09:30    DEBUG:   Bias Diff (mean, med, std) = 0.8, 0.8, 1.41
2019-10-14 15:09:34    DEBUG:   Bias Diff (mean, me

In [ ]:
dc